# Введение в ТВиМС: практикум по доверительным интервалам

*Тамбовцева Алла, НИУ ВШЭ*

## Доверительные интервалы

### Информация о данных

В рамках этого файла используем данные с результатами опроса преподавателей двух каталонских универсистетов (Открытый университет Каталонии и университет Помпеу Фабра), посвященного использованию Википедии в методических и академических целях.

* Codebook для набора данных: [ссылка](https://github.com/allatambov/PyDataAnalysis/blob/main/wiki_codebook.pdf).
* Оригинальное описание на платформе data.world: [ссылка](https://data.world/uci/wiki4he).

### Загрузка данных 

Импортируем библиотеку `pandas` с сокращенным названием `pd` (понадобится для работы с данными) и библиотеку `numpy` с сокращенным названием `np` (понадобится для более удобных вычислений):

In [1]:
import numpy as np
import pandas as pd

Загрузим данные из файла `"wiki.csv"` и сохраним их в датафрейм `wiki`, сообщив Python, что в качестве разделителя столбцов используется точка с запятой, а не запятая:

In [2]:
wiki = pd.read_csv("wiki.csv", sep = ";")

Если мы не добавим аргумент `sep` с разделителем, все данные склеятся в один столбец, так как по умолчанию функция `read_csv()` подразумевает разделение по запятой (ведь название формата CSV – это сокращение от *comma separated values*).

**Примечание для Jupyter Notebook.** Удобно, если файл с данными при работе лежит в той же папке, что и текущий ipynb-файл, в котором мы запускаем код, так не придется полностью прописывать к нему путь, достаточно одного названия с расширением.

**Примечание для Google Colab.** Загрузить файл с данными в облачное хранилище можно через кнопку *Files* (значок папки слева от рабочей области с ячейками), при нажатии на которую появляется возможность выбрать файл с компьютера (значок стрелки). После добавления файла его можно выбрать, кликнуть на три точки справа от названия, скопировать путь через *Copy path* и вставить его в функцию `read_csv()`. Например:

    wiki = pd.read_csv("/content/wiki.csv", sep = ";") 

Определим размерность датафрейма – число строк и число столбцов:

In [3]:
wiki.shape

(913, 53)

Итак, в датафрейме 913 строк и 53 столбца. Значит, всего было опрошено 913 человек.

### Доверительный интервал для доли

Вспомним, как вычисляется доверительный интервал для доли:

$$
\hat{p} - z^* \times se < p < \hat{p} + z^* \times se,
$$

где $p$ – доля в генеральной совокупности, которая нам неизвестна и которую мы хотим оценить с помощью доверительного интервала, $\hat{p}$ – доля, полученная по выборке, $z^*$ – z-значение для выбранного уровня доверия, $se$ – стандартное отклонение доли, тоже полученное по выборке, которое вычисляется так ($n$ – число наблюдений в выборке):

$$
se = \sqrt{\frac{\hat{p}(1-\hat{p})}{n}}.
$$

Давайте построим 95%-ный доверительный интервал для доли мужчин среди профессорско-преподавательского состава двух университетов Каталонии. Посмотрим на столбец со значениями пола:

In [4]:
wiki["GENDER"]

0      0
1      0
2      0
3      0
4      0
      ..
908    0
909    0
910    0
911    0
912    1
Name: GENDER, Length: 913, dtype: int64

Зафиксируем общее число наблюдений – атрибут `.size`:

In [5]:
n = wiki["GENDER"].size 
n

913

Мужской пол в данных закодирован единицей, поэтому для вычисления доли мужчин нам достаточно посчитать сумму всех значений (так как здесь только 0 и 1, она совпадает с числом единиц) и поделить на общее число наблюдений:

In [6]:
p = wiki["GENDER"].sum() / n  # доля p^
p

0.4249726177437021

Теперь, зная $n$ и $\hat{p}$, мы можем посчитать стандартное отклонение доли $s_{\hat{p}}$. Для извлечения квадратного корня возьмем функцию `sqrt()` из библиотеки `numpy`:

In [7]:
s_p = np.sqrt(p * (1 - p) / n) 
s_p

0.016360227864035466

Для построения интервала воспользуемся специальной функцией из модуля `stats` библиотеки `scipy` (от *scientific python*). Этот модуль часто импортируется с сокращенным названием:

In [8]:
import scipy.stats as st

Сама функция называется `norm.interval()`, она позволяет строить некоторые интервалы, предполагая, что те оценки, для которых мы эти интервалы строим, берутся из нормального распределения с некоторым средним (`loc`) и некоторым стандартным отклонением (`scale`). Можем подставить наши значения `p` и `s_p` в качестве параметров такого распределения и зафиксировать уровень доверия:

In [9]:
st.norm.interval(0.95, loc = p, scale = s_p)

(0.39290716035132395, 0.45703807513608025)

Итак, с 95% уверенностью можно считать, что истинная доля мужчин, работающих в университетах Каталонии, лежит в интервале от 0.39 до 0.46.

### Доверительный интервал для среднего

Допустим, мы хотим понять, в каком интервале, с 95%-ной уверенностью, лежит среднее значение возраста сотрудников двух каталонских университетов. 

Для начала посмотрим на описательные статистики столбца `AGE`:

In [10]:
wiki["AGE"].describe()

count    913.000000
mean      42.246440
std        8.058418
min       23.000000
25%       36.000000
50%       42.000000
75%       47.000000
max       69.000000
Name: AGE, dtype: float64

Итак, средний возраст примерно равен 42 годам, медианный тоже. Но, на самом деле, истинное среднее значение возраста может быть немного иным, исследователи же не смогли опросить всех сотрудников обоих университетов, их там много!

Чтобы учесть такую неопределенность в наших оценках, построим доверительный интервал для среднего. Вспомним, как он считается:

$$
\bar{x} - t^* \times se < \mu < \bar{x} + t^* \times se.
$$

Здесь $\mu$ – среднее генеральной совокупности, которое нам неизвестно и которое мы хотим оценить с помощью доверительного интервала, $\bar{x}$ – среднее, полученное по выборке, $t^*$ – t-значение, соответствующее выбранному уровню доверия, $se$ – стандартное отклонение среднего, тоже полученное по выборке, которое вычисляется на основе стандартного отклонения самой выборки $s$ и ее объема $n$:

$$
se = \frac{s}{\sqrt{n}}.
$$


Итак, опять соберем все составные части для формулы. 

In [11]:
n = wiki["AGE"].size  # размер выборки
mu = wiki["AGE"].mean()  # среднее выборки mu^
s = wiki["AGE"].std()  # ст отклонение выборки s

Вычислим стандартное отклонение среднего:

In [12]:
s_mu = s / np.sqrt(n)
s_mu

0.26669471758704505

Для построения доверительного интервала для среднего также воспользуемся специальной функцией из модуля `stats`. В случае с долей мы использовали функцию `norm.interval()`, так как там нам нужно было нормальное распределение, а здесь нам понадобится функция `t.interval()` для распределения Стьюдента. Помимо уровня доверия, значение `mu` и `s_mu` нам необходимо число степеней свободы для распределения Стьюдента, оно равно $\text{df} = n - 1$. Подставлям все и вычисляем:

In [13]:
st.t.interval(0.95, df = n - 1, loc = mu, scale = s_mu) 

(41.723033639899846, 42.7698469734627)

Итак, доверительный интервал получился довольно узким. С 95%-ной уверенностью можно утверждать, что средний возраст сотрудников каталонских университетов лежит в интервале от 42 до 43 лет (тут разумнее округлить значения до целых).